<a href="https://colab.research.google.com/github/priyadarshighosh/Study-ML-Everyday-/blob/main/ML_day29_titanic_non_pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt     #importing the basic data Science Libraries like numpy pandas matplotlib and seaborn
import seaborn as sns

from sklearn.model_selection import train_test_split               #for splitting the data into test and training data
from sklearn.compose import ColumnTransformer                       #for transforming the columns
from sklearn.impute import SimpleImputer                             #for imputing the missing values
from sklearn.preprocessing import OneHotEncoder                      #one hot encoding
from sklearn.preprocessing import MinMaxScaler                        #standard scaling
from sklearn.pipeline import Pipeline,make_pipeline                    #here we wont use pipelines
from sklearn.feature_selection import SelectKBest,chi2                 #feature selection

from sklearn.tree import DecisionTreeClassifier

In [86]:
from google.colab import files    # we are importing the file from the device
uploaded = files.upload()

Saving Titanic-Dataset.csv to Titanic-Dataset (3).csv


In [87]:
df=pd.read_csv('Titanic-Dataset.csv')   #fitting the data in the df dataframe
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [88]:
df.drop(['PassengerId', 'Name', 'Ticket' , 'Cabin'], inplace=True , axis=1)

In [89]:

X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,             #splitting the train and test data
                                                random_state=42)             #in the ratio 80 to 20

In [90]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [91]:
y_train.sample(5)

,Survived
523,1
589,0
184,1
236,0
1,1


x has data without the survived column and y has only the survived column
x_train has 80 perc of the total X data , X_test has 20 perc of the total x data
y_train has 80 perc of the total Y data , Y_test has 20 perc of the total y data

In [92]:
df.isnull().sum()        #finding tge null values in each feature

,0
Survived,0
Pclass,0
Sex,0
Age,177
SibSp,0
Parch,0
Fare,0
Embarked,2


this show that how much null values do we have in general like age has 177 null values and cabin has 687 null values . Its the total sum of it


In [93]:
#applying the imputer

si_age=SimpleImputer()                                           #imputer the age with the help of mean of ages
si_embarked=SimpleImputer(strategy='most_frequent')               #imputer the embarked with the most frequent value

X_train_age=si_age.fit_transform(X_train[['Age']])                            # fit tranform in training data
X_train_embarked=si_embarked.fit_transform(X_train[['Embarked']])             #  fitting the embarked data

X_test_age=si_age.transform(X_test[['Age']])                                    # fitting the data in the  testing data
X_test_embarked=si_embarked.transform(X_test[['Embarked']])

In [96]:
# one hot encoding                                   #we put ohe on categorical data

ohe_sex=OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_embarked=OneHotEncoder(sparse=False,handle_unknown='ignore')          #putting the ohe on the categorical data
                                                                           #in sex and embarked

X_train_sex=ohe_sex.fit_transform(X_train[['Sex']])                       #fitting data X_train and y_test
X_train_embarked=ohe_embarked.fit_transform(X_train_embarked)

X_test_sex=ohe_sex.transform(X_test[['Sex']])
X_test_embarked=ohe_embarked.transform(X_test_embarked)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [97]:
X_train.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
118,1,male,24.0,0,1,247.5208,C
536,1,male,45.0,0,0,26.5500,S
361,2,male,29.0,1,0,27.7208,C
29,3,male,NaN,0,0,7.8958,S
55,1,male,NaN,0,0,35.5000,S


In [98]:
X_train_embarked

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [99]:
X_train_rem = X_train.drop(columns=['Sex','Age','Embarked'])
X_test_rem= X_test.drop(columns=['Sex','Age','Embarked'])


X_train_transformed = np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_embarked),axis=1)
X_test_transformed = np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_embarked),axis=1)

In [100]:
X_train_transformed

array([[1., 0., 0., ..., 0., 0., 1.],
       [2., 0., 0., ..., 0., 0., 1.],
       [3., 0., 0., ..., 0., 0., 1.],
       ...,
       [3., 2., 0., ..., 0., 0., 1.],
       [1., 1., 2., ..., 0., 0., 1.],
       [1., 0., 1., ..., 0., 0., 1.]])

In [101]:
from sklearn.tree import DecisionTreeClassifier

In [102]:
clf= DecisionTreeClassifier()
clf.fit(X_train_transformed, y_train)

DecisionTreeClassifier()

In [107]:
X_train_transformed.shape

(712, 10)

In [108]:
X_test_transformed.shape

(179, 10)

In [109]:
X_train_transformed


array([[1., 0., 0., ..., 0., 0., 1.],
       [2., 0., 0., ..., 0., 0., 1.],
       [3., 0., 0., ..., 0., 0., 1.],
       ...,
       [3., 2., 0., ..., 0., 0., 1.],
       [1., 1., 2., ..., 0., 0., 1.],
       [1., 0., 1., ..., 0., 0., 1.]])

In [111]:
X_test_transformed

array([[3., 1., 1., ..., 1., 0., 0.],
       [2., 0., 0., ..., 0., 0., 1.],
       [3., 0., 0., ..., 0., 0., 1.],
       ...,
       [3., 1., 5., ..., 0., 0., 1.],
       [2., 0., 0., ..., 0., 0., 1.],
       [3., 1., 1., ..., 0., 0., 1.]])

In [115]:
y_pred=clf.predict(X_test_transformed)

In [116]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7932960893854749